# Load packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# color-blind color scheme
plt.style.use('tableau-colorblind10')

Let's make sure that we are using numpy 1.26.4 for backwards compatibility (last version before 2.0, which doesn't yet have support from all packages):

In [ ]:
np.version.version

# A simple LIF neuron

We'll define a simple leaky-integrate-and-fire neuron model with leaky integration of a current $I$.

This implementation is loosely based on:
https://colab.research.google.com/github/johanjan/MOOC-HPFEM-source/blob/master/LIF_ei_balance_irregularity.ipynb#scrollTo=Hhk7e-QreVSh







## LIF dynamcis

One neuron follows the LIF dynamcis

$C_m \frac{dv}{dt} = - g_l [v(t) - V_l] + I$,

where $v(t)$ is the membrane voltage, $C_m$ is the membrane capactitance, $g_l$ the leak conductance, $V_l$ the leak potential.


Dividing by $g_l$, we rewrite this more simply as

$\tau_m \frac{dv}{dt} = - [v(t) - V_l] + I/g_l $, where $\tau_m \equiv C_m/g_l $.

Because we can only simulate discrete time steps, we estimate the derivative of $v(t)$ by the Euler forward method:

$\frac{dv}{dt} \mapsto $ `dv / dt` : now, instead of representing the derivative, we have two variables `dv` and `dt`, representing discrete voltage and time differences.

__Note that the Euler method carries hidden assumptions, which can break down and distort results.__

But for now, we should be fine. The discrete time dynamics we will implement are thus:

`dv = {- [v - V_l] + I/g_l } / \tau_m * dt`


### Spiking mechanism



The spiking mechanism is simply implemented by the condition

if $v(t) \geq V_{th}$, then $v(t + dt) = V_{reset}$.

We also implement a refractory period, by adding a counter variable since the last spike. For every time step:

- if neuron currently spiking: `refractory_counter` = `tau_ref/dt` (which expresses $\tau_{ref}$ in terms of time steps)
- if neuron is refractory: clamp voltage to `V_{reset}` and decrease `refractory_counter` by one


## Code

The whole looks like this:

In [ ]:
class LIF_neuron:
    # initialize a neuron class
    # provided parameter dictionary params
    def __init__(self, params):
        # attach parameters to object
        self.V_th, self.V_reset = params['V_th'], params['V_reset']   
        self.tau_m, self.g_L = params['tau_m'], params['g_L']        
        self.V_init, self.V_L = params['V_init'], params['V_L']       
        self.dt = params['dt']
        self.tau_ref = params['tau_ref']

        # initialize voltage and current
        self.v = 0.0
        # time steps since last spike
        self.refractory_counter = 0
    
    def LIF_step(self, I):
        """
            Perform one step of the LIF dynamics
        """
        
        currently_spiking = False
        
        if self.refractory_counter > 0:
            # if the neuron is still refractory
            self.v = self.V_reset
            self.refractory_counter = self.refractory_counter - 1
        elif self.v >= self.V_th:
            # if v is above threshold,
            # reset voltage and record spike event
            currently_spiking = True
            self.v = self.V_reset
            self.refractory_counter = self.tau_ref/self.dt
        else:
            # else, integrate the current:
            # calculate the increment of the membrane potential
            dv = self.voltage_dynamics(I)
            # update the membrane potential
            self.v = self.v + dv

        return self.v, currently_spiking
    
    def voltage_dynamics(self, I):
        """
            Calulcates one step of the LI dynamics
        """
        dv = (-(self.v-self.V_L) + I/self.g_L) * (self.dt/self.tau_m)
        return dv
        

You can see that we have split up the functions implementing the dynamics into `LIF_step` and `voltage_dynamics`. We could have put both into the same function, but this will be useful later.

In [ ]:
params = {}
### typical neuron parameters###
params['V_th']    = -55. # spike threshold [mV]
params['V_reset'] = -75. #reset potential [mV]
params['tau_m']   = 10. # membrane time constant [ms]
params['g_L']     = 10. #leak conductance [nS]
params['V_init']  = -65. # initial potential [mV]
params['V_L']     = -75. #leak reversal potential [mV]
params['tau_ref']    = 2. # refractory time (ms)
params['dt'] = .1  # Simulation time step [ms]

In [ ]:
# initialize one neuron, i.e. an instance of the class
neuron1 = LIF_neuron(params)

In [ ]:
# let's check if everything works by performing one step:
neuron1.LIF_step(I=300.0)

In [ ]:
# simulate 500 time steps
voltages = []
spikes = []
for _ in range(500):
    v, s = neuron1.LIF_step(I=300.0)
    voltages.append(v)
    spikes.append(s)

In [ ]:
plt.figure()
plt.plot(voltages)
# for s in np.where(spikes)[0]:
#     plt.axvline(s, c='red')
plt.xlabel('Time step')
plt.ylabel('V (mV)');
plt.show()

Neat, so we have a single LIF neuron that spikes regularly.

# Overriding classes with __super__

Let's say that now we want to implement a different class of neurons: [exponential LIF](https://en.wikipedia.org/wiki/Exponential_integrate-and-fire), which have modified dynamics:

$C_m \frac{dv}{dt} = - g_l [v(t) - V_l - \Delta_T \exp\Big( \frac{v(t) - V_{trig}}{\Delta_T} \Big)] + I$.

Instead of firing once $V_{tr}$ is reached, the new term $\Delta_T \exp\Big( \frac{v(t) - V_{trig}}{\Delta_T}\Big)$ starts growing quickly as $v(t)$ approaches $V_{trig}$, modeling a more realistic firing behaviour.

So now, we can set $V_{trig}$ to -55 mV, and the old threshold $V_{th}$ has the new function of acting as a separate reset, which we will set to 0 mV.


## Code

To implement this, we could now copy and modify the `LIF_neuron` class. But as models build on eachother, there is a more efficient way of doing this:

We can initialize the class `ExpLIF_neuron` as a child of `LIF_neuron`, inheriting all of its attributes and functions. The initialization can be inherited by using the `super()` function:

In [ ]:
# define new class as child of old class
class ExpLIF_neuron(LIF_neuron):
    def __init__(self, params):
        # build on LIF neuron with same settings
        # (this will run __init__ of the parent class)
        super().__init__(params)
        
        # we only need to attach additional variables:
        self.DeltaT = params['DeltaT']
        self.V_exp_trigger = params['V_exp_trigger']
    
    # now we can just    
    def voltage_dynamics(self, I):
        """
            Calulcates one step of the exp-LI dynamics
        """
        dv = (-(self.v-self.V_L) + I/self.g_L + self.DeltaT * np.exp((self.v-self.V_exp_trigger)/self.DeltaT)) * (self.dt/self.tau_m)
        return dv
        

So that's why we separated `voltage_dynamics` into its own function: because we knew that we will implement a child class that will modify it, but inherit all other properties:

*When you write a class, you want other classes to be able to use it.*
*super() makes it easier for other classes to use the class you're writing.*

*As Bob Martin says, a good architecture allows you to postpone decision making as long as possible.*

*super() can enable that sort of architecture.* [From SO](https://stackoverflow.com/questions/222877/what-does-super-do-in-python-difference-between-super-init-and-expl)

(the problem can be that your base class becomes so generic, it is hard to understand what each function does, but that can be fixed by good documentation)

In [ ]:
# additional parameters for ExpLIF neurons
params['DeltaT'] = 10.0  # sharpness of exponential peak
params['V_exp_trigger'] = -55. # threshold for exponential depolarization [mV]
params['V_th'] = 0 # new reset threshold [mV]

In [ ]:
# initialize one neuron
neuron2 = ExpLIF_neuron(params)

In [ ]:
# simulate 500 time steps
voltages = []
spikes = []
for _ in range(500):
    v, s = neuron2.LIF_step(I=300.0)
    voltages.append(v)
    spikes.append(s)

In [ ]:
plt.figure()
plt.plot(voltages)
# for s in np.where(spikes)[0]:
#     plt.axvline(s, c='red')
# plt.xlim(0, 100)
plt.xlabel('Time step')
plt.ylabel('V (mV)');
plt.show()

Great! So now we have a modified class which reproduces expLIF dynamics.

## Adaptive exponential LIF neuron (AdEx)

To model biology more closely, we can implement adaptation. This accounts for more diverse neuronal firing patterns, such as adaptation, bursting and initial bursting.

Using the same approach as above, implement the following dynamics of AdEX neurons:

$C_m \frac{dv}{dt} = - g_l [v(t) - V_l - \Delta_T \exp\Big( \frac{v(t) - V_{trig}}{\Delta_T} \Big)] + I - \omega(t)$,

with the adaptation variable $\omega$. The dynamics of $\omega$ are determined by two parts:

- subthreshold adaptation, which follows $\tau_\omega \frac{d\omega}{dt} = a [v(t)-V_l] - \omega$.
- a spike-triggered adaptation term, which increases $\omega$ by $b$ at every spike: if $v(t) \geq V_{th}$, then $\omega(t + dt) = \omega(t) + b$.

$\tau_\omega$ is the time scale of the adaptation variable; $a$ is the subthreshold adaptation parameter; $b$ is the spike-triggered adaptation parameter.

See https://neuronaldynamics.epfl.ch/online/Ch6.S1.html for further explanations on the model parameters.

In [ ]:
# Implement an AdEx neuron class using the ExpLIF parent class
class AdEx_neuron(ExpLIF_neuron):
    def __init__(self, params):
        ...